The dataset can be downloaded [here](https://github.com/zygmuntz/goodbooks-10k). 

In [2]:
import pandas as pd
books = pd.read_csv('books.csv')

In [3]:
books = books[['goodreads_book_id', 'title']]
books = books.rename(columns = {'goodreads_book_id':'book_id'})
books = books.set_index('book_id')

In [4]:
url = 'https://www.goodreads.com/book/show/'

In [5]:
books['summary'] = "" 

In [ ]:
import pickle
import time
import requests
from bs4 import BeautifulSoup

In [ ]:
while True:
    book_ids = list(books[books.summary == ""].index.unique())
    
    print('\n\n\n\n books left: ', len(book_ids), '\n') #This tells us how many book summaries are left to scrape.
    
    if len(book_ids) <= 45:
        break

    for index, book_id in enumerate(book_ids):
        #We pickle the dataframe every 200 iterations and stop running for 60 seconds, to avoid getting 
        #blocked by Goodreads
        if index % 200 == 199:
            with open('book_summaries.pkl', 'wb') as picklefile:
                pickle.dump(books, picklefile)
            time.sleep(60)
            
        try: 
            request = requests.get(url+str(book_id))
            text = request.text
            soup = BeautifulSoup(text, "lxml")
            summary = soup.find("div", {"id": "description"}).find_all('span')[-1].text
            books.summary.loc[book_id] = summary
            print(book_id)
            
        except: 
            continue
        
    with open('book_summaries.pkl', 'wb') as picklefile:
        pickle.dump(books, picklefile)
            